# BLU06 - Learning Notebook - Data wrangling workflows - Part 3 of 3

In [1]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

from category_encoders.ordinal import OrdinalEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# 3 Advanced pipelines in scikit-learn

Remember our workflow diagram? Let's look at it again.

![data_transformation_workflow](../media/data_processing_workflow.png)

*Fig 1. - A standard workflow (again).*

Pandas, as amazing as it is, can only take us fo far.

There, beyond the known universe, lies **modeling**.

Where we are at this point:
* We are to perform transformations on data, setting up robust pipelines using nothing but Pandas
* We are able to combine different dataframes, to enrich our datasets or generate new ones.

Thus, here we are, modeling lying ahead of us. What's exactly new about modeling though?

We will be using the same dataset, but this time we will create a train-test split, as we would do before modeling.

In [2]:
works = pd.read_csv('../data/works.csv')
X_train, X_test = train_test_split(works)

## 3.1 How is modeling different from transformation

In Pandas, we simply transformed the original dataframe into a new one.

But sometimes, this isn't possible. Let's start with an example: encoding categorical variables.

Remember: we need to perform the exact same transformations on train and test data (and whatever data comes next).

In [3]:
def transform_data(df):
    df = df.copy()
    df = (df.pipe(remove_intervals)
            .pipe(label_encoder, 'ComposerName'))
    return df


def remove_intervals(df):
    df = df.copy()
    mask = df['Interval'].isnull()
    df = (df.loc[mask, :]
            .drop(columns='Interval'))
    return df
    

def label_encoder(df, column):
    df = df.copy()
    df[column + 'Encoded'] = df[column].astype('category').cat.codes
    return df


X_train_ = transform_data(X_train)

train_alban_berg = X_train_['ComposerName'] == 'Berg,  Alban'
(X_train_.loc[train_alban_berg, ['ComposerName', 'ComposerNameEncoded']]
         .drop_duplicates())

,ComposerName,ComposerNameEncoded
11790,"Berg, Alban",167


All is good. We removed the intermissions (just like we did previously) and we transformed the original dataframe.

For convenience, we are keeping only the `ComposerName` and `ComposerNameEncoded` columns and removing duplicates.

Let's do the same to the test data.

In [4]:
X_test_ = transform_data(X_test)

test_alban_berg = X_test_['ComposerName'] == 'Berg,  Alban'
(X_test_.loc[test_alban_berg, ['ComposerName', 'ComposerNameEncoded']]
        .drop_duplicates())

,ComposerName,ComposerNameEncoded
44313,"Berg, Alban",109


Do you see the problem? The same `ComposerName` can (and will, in all probability) get different encodings.

This is no minor problem, as it would lead us to make totally wrong predictions!!

There are other cases in which problems arise. For instance, when replacing missing values with the mean:
* You are supposed to compute the mean on the training set and use it to transform both train and test sets
* Otherwise, you end up underestimating your true generalization error.

This particular unit is not about modeling at a conceptual level, but you get the point: 
* Somehow, you need to fit the transformer on your training data first (e.g., define the encodings, compute the means)
* Transform both train and test sets (and any data that might come in, really) using the pre-fitted transformers.

This is more akin to modeling. In fact, all of this *is* modeling and part of your model. 

How do we solve this? **We need transformers that are more like models.**

## 3.2 Meet the sklearn-like transformers

There are at three fundamental verbs in scikit-learn and sklearn-like libraries:
* `.fit()`
* `.transform()`
* `.predict()`.

You are already familiar with `.fit()` and `.predict()`, from the Bootcamp and the Hackathon #1.  We use them to train models and make predictions.

Here, we will explore a new combo: `.fit()` and `.transform()`. This is how it works.

![sklearn_like_transformation_pipeline](../media/sklearn_like_transformation_pipeline.png)

*Fig 2. - A data pipeline with consistent transformers, fitted on the training set.*

In short, we fit a transformer on the training data and use to transform the training data.

We will, however, return the transformer so we can use it to transform new, incoming data as well. Confusing? Perhaps.

Time to get practical: meet the `categorical_encoders`, a set of transformers for encoding categorical variables.

In [5]:
encoder = OrdinalEncoder(cols=['ComposerName'])
X_train_ = encoder.fit_transform(X_train)
X_train_.head()

,GUID,ProgramID,WorkID,MovementID,WorkTitle,Movement,ConductorName,Interval,ComposerName
45019,15b58e17-7216-4c00-a362-b6eeb534cab6,11369,5335,NaN,COTILLON,NaN,"Smallens, Alexander",NaN,1
16890,d89bb70e-6d0f-4b37-b3b4-ec7f72baa68d,4323,0,NaN,NaN,NaN,NaN,Intermission,2
4942,e9176b27-33cb-479d-97bc-9aa7ec4c4b68,1866,856,NaN,"ROSAMUNDE: OVERTURE, D.644",NaN,"Barbirolli, John",NaN,3
51184,5aaeb545-f7b2-4a0a-b0ff-a7b568c101a4,9798,50064,NaN,"SYMPHONY NO. 4, F MINOR, OP. 36",NaN,"Maazel, Lorin",NaN,4
47209,5d851fd7-2168-4f53-8c15-5c69b0e2a568,1250,52942,NaN,SYMPHONY NO. 6,NaN,"Bernstein, Leonard",NaN,5


We can now simply use transform in our test set. Pretty convenient, huh?

In [6]:
X_test_ = encoder.fit_transform(X_test)
X_test_.head()

,GUID,ProgramID,WorkID,MovementID,WorkTitle,Movement,ConductorName,Interval,ComposerName
74728,752e1ae0-bf6b-4349-ab84-ea27da385b19,443,3630,NaN,"BOEUF SUR LE TOIT, OP 58",NaN,"Kostelanetz, Andre",NaN,216
59611,e3fb6f6d-93c3-456e-a1b4-5c1fb368f00e,7864,6994,NaN,VIOLIN CONCERTO,NaN,"Slatkin, Leonard",NaN,127
54625,2cbb7c9b-addc-4ce8-9b98-05a54737179c,12802,9006,17.0,GOTTERDAMMERUNG [GÖTTERDÄMMERUNG],PROLOGUE: Siegfried's Rhine Journey,"Hoogstraten, Willem van",NaN,12
26689,ddc76305-d77a-4528-b812-53c4068b707d,11900,51071,NaN,"CONCERTO, PIANO NO. 25, C MAJOR, K.503",NaN,"Gilbert, Alan",NaN,85
8458,4d34498f-d5ec-4f87-9176-afda04827a50,6387,53193,NaN,"SORCERER'S APPRENTICE, THE",NaN,"Kostelanetz, Andre",NaN,152


Let's re-do our transformer functions, so that they can either fit a transformer or accept a pre-fitted one.

We have to change our `label_encoder()` first to incorporate this logic. Then we need to adapt `transform_data()`.

In [7]:
def transform_data(df, encoder=None):
    df = df.copy()
    df = (df.pipe(remove_intervals)
            .pipe(label_encoder, 'ComposerName', encoder))
    
    return df


def label_encoder(df, columns, encoder=None):
    if encoder:
        preview_encodings(encoder)
        return encoder.transform(df)
    else:
        encoder = OrdinalEncoder(cols=[columns])
        df = df.copy()
        df = encoder.fit_transform(df)
        preview_encodings(encoder)
        return df, encoder

    
def preview_encodings(encoder):
    encodings = encoder.category_mapping[0]['mapping'][:4]
    print('Encodings: {}'.format(encodings))
    return None

X_train_, encoder = transform_data(X_train)

Encodings: [('Benjamin,  Arthur', 1), ('Schubert,  Franz', 2), ('Tchaikovsky,  Pyotr  Ilyich', 3), ('Schuman,  William', 4)]


In the code above, we changed our functions so that they can receive an encoder. Otherwise, they fit a new one and return it to be re-used.

From a consistency standpoint, things should be looking good. Nonetheless, you are previewing the encoder as a sanity check.

In [8]:
X_test_ = transform_data(X_test, encoder=encoder)

Encodings: [('Benjamin,  Arthur', 1), ('Schubert,  Franz', 2), ('Tchaikovsky,  Pyotr  Ilyich', 3), ('Schuman,  William', 4)]


What kind of transformations do you need to perform this way? Some very common ones are:
* Encoding (as we've seen)
* Scaling
* Vectorization (you will learn about this in the next specialization!)
* Missing data imputation.

Now, this clearly changes things (right?):
* We lose the ability to do method chaining, as we have to return encodings as intermediate outputs
* We need to segregate pipelines for training (fit and transform) and test (transform), which adds complexity and it's error prone.

Because we will perform the exact same transformations on all datasets, storing all the exact steps is key for reproducibility and consistency.

Turns out, scikit-learn provides us with a special take on pipelines, to wrap all of this in a single META-TRANSFORMER.

![megazord](../media/megazord.png)

*Fig 3. - A meta-transformer in practice.*

Meet the Megazord.

## 3.3 Pipelines

The sklearn's `Pipelines` provide a higher level of abstraction than the individual building blocks.

Let's tie together all these sequential transformers and run `Megazord.fit()` and `Megazod.transform()` on the whole thing.

That would make managing our code much easier, right? Let's do it:
* We want to replace the missing values with the mode
* We want to one-hot-encode all categorical variables.

First things first, some Pandas magic: let's drop the ID columns and exclude the intervals.

In [9]:
def prepare_data(df):
    df = df.copy()
    df = (df.pipe(drop_id_fields)
            .pipe(remove_intervals)
            .drop_duplicates())
    return df

def drop_id_fields(df):
    columns = ['GUID', 'ProgramID', 'WorkID', 'MovementID']
    df = df.copy()
    df = df.drop(columns=columns)
    return df

X_train_ = prepare_data(X_train)
X_train_.head()

,ComposerName,WorkTitle,Movement,ConductorName
45019,"Benjamin, Arthur",COTILLON,NaN,"Smallens, Alexander"
4942,"Schubert, Franz","ROSAMUNDE: OVERTURE, D.644",NaN,"Barbirolli, John"
51184,"Tchaikovsky, Pyotr Ilyich","SYMPHONY NO. 4, F MINOR, OP. 36",NaN,"Maazel, Lorin"
47209,"Schuman, William",SYMPHONY NO. 6,NaN,"Bernstein, Leonard"
2993,"Strauss, Richard","MILITARMARSCHE, OP. 57",NaN,"Hoogstraten, Willem van"


In [10]:
X_test_ = prepare_data(X_test)
X_test_.head()

,ComposerName,WorkTitle,Movement,ConductorName
74728,"Milhaud, Darius","BOEUF SUR LE TOIT, OP 58",NaN,"Kostelanetz, Andre"
59611,"Adams, John",VIOLIN CONCERTO,NaN,"Slatkin, Leonard"
54625,"Wagner, Richard",GOTTERDAMMERUNG [GÖTTERDÄMMERUNG],PROLOGUE: Siegfried's Rhine Journey,"Hoogstraten, Willem van"
26689,"Mozart, Wolfgang Amadeus","CONCERTO, PIANO NO. 25, C MAJOR, K.503",NaN,"Gilbert, Alan"
8458,"Dukas, Paul","SORCERER'S APPRENTICE, THE",NaN,"Kostelanetz, Andre"


Here's how we are going to do:
1. Create an `Imputer` that replace missing values with the mode of the column
2. Use said `Imputer` to impute the missing values
3. Ordinal encoding all categorical features.

All in one, single, amazing, Megazord.

## 3.4 Custom transformers

We can build own custom transformers, for as long as they follow the usual blueprint:
* Implement `Transformer.fit()`
* And `Transformer.transform()`.

All scikit-learn estimators have `get_params()` and `set_params` functions. 

The easiest way to implement these functions in a sensible way is to inherit from `sklearn.base.BaseEstimator`, as we're doing.

And `Pipeline` compatibility requires a `fit_transform()` method that we are inheriting from `sklearn.base.TransformerMixin`.

In [11]:
class FeatureMultiplier(BaseEstimator, TransformerMixin):
    def __init__(self, some_parameter):
        self.some_parameter = some_parameter

    def fit(self, X, y=None):
        # Fit the transformer and store it.
        return self
        
    def transform(self, X):
        # Transform X.
        return X

Also, we may want our transformer to accept some parameters.

That's we are doing when we include `some_parameter` in the `__init__`.

Back to our transformers. Our blueprint:
* We want out estimator to be able to take a `strategy` parameter, although we will support only the mode
* Fitting requires taking the mode of each column and storing it
* Transform implies replacing missing values with the given column modes.

How are we going to compute the modes? Pandas, as always, provides a convenient `.mode()` method.

In [12]:
X_train_.mode()

,ComposerName,WorkTitle,Movement,ConductorName
0,"Beethoven, Ludwig van",MESSIAH,Overture,"Damrosch, Walter"


To be able to use indexing we will use `df.squeeze()`, a convenient method to transform our dataframe into a `pd.Series`.

In [13]:
X_train.mode().squeeze()

GUID             f549e93f-b35f-4824-b0d5-d543953535f8
ProgramID                                        9542
WorkID                                              0
MovementID                                          1
ComposerName                         Wagner,  Richard
WorkTitle                  MEISTERSINGER, DIE, WWV 96
Movement                                     Overture
ConductorName                        Damrosch, Walter
Interval                                 Intermission
Name: 0, dtype: object

We have everything we need. Let's do this!

In [14]:
class CategoryImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy=None):
        if strategy:
            self.strategy = strategy
        else:
            self.strategy = 'most_frequent'

    def fit(self, X, y=None):
        if self.strategy == 'most_frequent':
            self.fills = X.mode(axis=0).squeeze()
            return self
        else:
            return 'Strategy not supported.'

    def transform(self, X):
        return pd.DataFrame(X).fillna(self.fills)


imputer = CategoryImputer()
X_train_ = imputer.fit_transform(X_train_)
X_train_.head()

,ComposerName,WorkTitle,Movement,ConductorName
45019,"Benjamin, Arthur",COTILLON,Overture,"Smallens, Alexander"
4942,"Schubert, Franz","ROSAMUNDE: OVERTURE, D.644",Overture,"Barbirolli, John"
51184,"Tchaikovsky, Pyotr Ilyich","SYMPHONY NO. 4, F MINOR, OP. 36",Overture,"Maazel, Lorin"
47209,"Schuman, William",SYMPHONY NO. 6,Overture,"Bernstein, Leonard"
2993,"Strauss, Richard","MILITARMARSCHE, OP. 57",Overture,"Hoogstraten, Willem van"


In [15]:
X_train_.isnull().sum()

ComposerName     0
WorkTitle        0
Movement         0
ConductorName    0
dtype: int64

There we go! What about the test set?

In [16]:
X_test_ = imputer.transform(X_test_)
X_test_.isnull().sum()

ComposerName     0
WorkTitle        0
Movement         0
ConductorName    0
dtype: int64

Victory awaits!

## 3.5 Everything together

Now, we want to fill in missing values and use one-hot-encoding (remember?), all at the same time.

We are reaching our destination!

In [17]:
megazord = Pipeline([('fill_na', CategoryImputer(strategy='most_frequent')),
                     ('encode', OrdinalEncoder())])

X_train_ = megazord.fit_transform(X_train)
X_test_ = megazord.transform(X_test)

This is it! How easier can we get? This way we abstract all the logic of passing transformers around.

Now, can we throw a model in there? Perhaps we can.

(But we shouldn't, in a way.)

In [18]:
megazord = Pipeline([('fill_na', CategoryImputer(strategy='most_frequent')),
                     ('encode', OrdinalEncoder()),
                     ('k_means', KMeans())])

megazord.fit(X_train_)
megazord.predict(X_test_)

array([4, 7, 7, ..., 2, 2, 5], dtype=int32)

Please note that we are exemplifying data wrangling workflows.

For the sake of simplicity, we are encoding categorical variables as if they were ordinal, instead of using one-hot-encoding, as recommended.

Take this for what it is: an example on how to build end-to-end pipelines for modelin in scikit-learn.